In [1]:
import json, os, re, random
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
from opencc import OpenCC
cc_t2s = OpenCC('t2s')
cc_s2t = OpenCC('s2t')

/home/zchen/miniconda3/envs/text_style_transfer/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [8]:
path = "/home/zchen/encyclopedia-text-style-transfer/tools/ckip/"
data_utils.download_data_url(path)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

path = path + "data"
ws = WS(path, disable_cuda=False)
pos = POS(path, disable_cuda=False)
ner = NER(path, disable_cuda=False)

2022-10-31 04:00:56.940540: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22307 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:17:00.0, compute capability: 8.6
2022-10-31 04:01:00.382183: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22307 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:17:00.0, compute capability: 8.6
2022-10-31 04:01:03.860793: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22307 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:17:00.0, compute capability: 8.6


In [4]:
def load_json(file):
    with open(file, 'r', encoding="utf-8") as f:
        return json.load(f)

def save_json(file, obj):
    with open(file, 'w', encoding="utf-8") as f:
        json.dump(obj, f, ensure_ascii=False)

In [9]:
file = "/home/zchen/encyclopedia-text-style-transfer/data/ETST/wiki_baidu_articles_pairs.json"
art_pairs = load_json(file)
print(len(art_pairs))

In [11]:
def get_baidu_sents_str_from_pairs(pairs):
    sents = []
    for src, targs in pairs:
        sents += [targ for targ, score in targs]
    return '\n'.join(sents)

def arts2sents_str(arts):
    return '\n'.join(list(arts.values()))
    
def get_s_t(s):
    s_s, s_t = cc_t2s.convert(s), cc_s2t.convert(s)
    s_s = [sent for sent in s_s.split('\n') if len(sent) > 5]
    s_t = [sent for sent in s_t.split('\n') if len(sent) > 5]
    assert len(s_s) == len(s_t), (len(s_s), len(s_t))
    
    zipped = list(zip(s_s, s_t))
    zipped = list(set(zipped))
    s_s, s_t = list(zip(*zipped))
    assert len(s_s) == len(s_t), (len(s_s), len(s_t))
    return s_s, s_t
    
def sents2ents(sentence_list):
    word_sentence_list = ws(
        sentence_list,
        # sentence_segmentation = True, # To consider delimiters
        # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
        # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
        # coerce_dictionary = dictionary2, # words in this dictionary are forced
    )
    pos_sentence_list = pos(word_sentence_list)
    entity_sentence_list = ner(word_sentence_list, pos_sentence_list)
    
    ent_sents = []
    for sent, word_sent, pos_sent, ent_sent in zip(sentence_list, word_sentence_list, pos_sentence_list, entity_sentence_list):
        ents = [word for word, pos in zip(word_sent, pos_sent) if re.match(r"N[a-d]", pos)]
        # re.match() determines whether the pattern matches the beginning of the string,
        # returns the matching object if it matches, and returns None if it does not match.
        
        ents += [ent[3] for ent in ent_sent]
        ent_sents.append(set(ents))
    return ent_sents
    
def get_score(src_ent_sent, targ_ent_sent):
    intersection_len = len(src_ent_sent & targ_ent_sent)
    if intersection_len == 0:
        return 0
    precision = intersection_len / len(targ_ent_sent)
    recall = intersection_len / len(src_ent_sent)
    targ_len = len(targ_ent_sent)
    return (2*precision*recall) / (precision+recall)
    
def filter_sent(zipped, p):
    """ filtering strategy, e.g., highest k sents or a threshold score p """
#     return zipped[:k]
    
    sents = []
    for sent, score in zipped:
        if score >= p:
            sents.append((sent, score))
        else:
            break
    return sents

def get_pair(src_sent, targ_sents, src_ent_sent, targ_ent_sents):
    scores = [get_score(src_ent_sent, targ_ent_sent) for targ_ent_sent in targ_ent_sents]
    zipped = list(zip(targ_sents, scores))
    zipped.sort(key=lambda x: x[1], reverse=True)
    matched = filter_sent(zipped, p=0.3)
    return (src_sent, matched)

def get_pairs(src_sents_str, targ_sents_str):
    src_sents_s, src_sents_t = get_s_t(src_sents_str)
    targ_sents_s, targ_sents_t = get_s_t(targ_sents_str)
    print(len(src_sents_s), len(targ_sents_s))
    src_ent_sents, targ_ent_sents = sents2ents(src_sents_t), sents2ents(targ_sents_t)
    return [get_pair(src_sent, targ_sents_s, src_ent_sent, targ_ent_sents) for src_sent, src_ent_sent in zip(src_sents_s, src_ent_sents)]

sent_pairs_w2b, sent_pairs_b2w = [], []
"""
[
    (src_sent_1, [(targ_sent_n, score), ...]),
    ...
    ]
"""

total = len(art_pairs)
for i, (title, (wiki_arts, baidu_arts)) in enumerate(art_pairs.items()):
    wiki_sents_str, baidu_sents_str = arts2sents_str(wiki_arts), arts2sents_str(baidu_arts)
    wiki2baidu = get_pairs(wiki_sents_str, baidu_sents_str)    # src: wiki, targ: baidu
    baidu2wiki = get_pairs(baidu_sents_str, wiki_sents_str)    # src: baidu, targ: wiki
    sent_pairs_w2b += wiki2baidu
    sent_pairs_b2w += baidu2wiki
    print(f"({i + 1} / {total})")

115 98
98 115
(1 / 44)
177 352
352 177
(2 / 44)
62 5
5 62
(3 / 44)
22 11
11 22
(4 / 44)
110 104
104 110
(5 / 44)
13 83
83 13
(6 / 44)
13 104
104 13
(7 / 44)
29 70
70 29
(8 / 44)
131 174
174 131
(9 / 44)
22 33
33 22
(10 / 44)
79 154
154 79
(11 / 44)
62 25
25 62
(12 / 44)
4 10
10 4
(13 / 44)
28 105
105 28
(14 / 44)
43 19
19 43
(15 / 44)
18 114
114 18
(16 / 44)
67 101
101 67
(17 / 44)
15 5
5 15
(18 / 44)
224 205
205 224
(19 / 44)
13 3
3 13
(20 / 44)
20 39
39 20
(21 / 44)
4 32
32 4
(22 / 44)
78 40
40 78
(23 / 44)
32 28
28 32
(24 / 44)
10 61
61 10
(25 / 44)
91 2
2 91
(26 / 44)
33 55
55 33
(27 / 44)
19 137
137 19
(28 / 44)
98 111
111 98
(29 / 44)
137 146
146 137
(30 / 44)
36 73
73 36
(31 / 44)
19 35
35 19
(32 / 44)
17 140
140 17
(33 / 44)
76 401
401 76
(34 / 44)
18 47
47 18
(35 / 44)
102 13
13 102
(36 / 44)
17 24
24 17
(37 / 44)
51 43
43 51
(38 / 44)
37 66
66 37
(39 / 44)
68 113
113 68
(40 / 44)
14 2
2 14
(41 / 44)
147 79
79 147
(42 / 44)
6 10
10 6
(43 / 44)
2 29
29 2
(44 / 44)


In [13]:
file = "/home/zchen/encyclopedia-text-style-transfer/data/ETST/sents_pairs_b2w.json"
save_json(file, sent_pairs_b2w)
file = "/home/zchen/encyclopedia-text-style-transfer/data/ETST/sents_pairs_w2b.json"
save_json(file, sent_pairs_w2b)

In [12]:
n_pairs = sum([len(pair) for pair in sent_pairs_w2b])
avg_pairs = n_pairs / len(sent_pairs_w2b)
print(n_pairs, avg_pairs)
print(sent_pairs_w2b[:10])

4758 2.0
[('金门县市区公车由金门县县营事业机构金门县公共车船管理处营运 ，有四个营运车站：金城车站、山外车站、沙美车站、烈屿车站。总计有29线编有编号的一般公车路线，以及5条台湾好行观光公车路线。', []), ('金门县辖有三镇（金城镇、金湖镇、金沙镇）、三乡（金宁乡、烈屿乡、乌坵乡（代管））。', [('金门县辖3个镇、2个乡，分别是金城镇、金湖镇、金沙镇、金宁乡、烈屿乡。', 0.6666666666666666), ('1954年6月，台当局在其控制的莆田县乌丘屿设置乌丘乡（乌丘村），由于莆田县主体已全部由中国大陆统治，故指定暂由金门县代管，此时金门县辖金城镇、金沙镇、金宁乡、金湖乡、金山乡、烈屿乡、乌丘乡（乌丘村）等二镇五乡。', 0.3888888888888889)]), ('另有代管的乌坵乡二岛屿（大坵与小坵），则地处东经119度28分，北纬24度59分，位在中华人民共和国福建省莆田外海。离金门本岛相距72海浬，大约位于金门与马祖中心点。', []), ('金门县县长是金门县政府之行政首长，负责综理县政，并指挥、监督所属职员及机构。现任金门县长杨镇浯为中国国民党籍第7届金门县县长。', []), ('民间文学方面的著作及编辑，有：林永塘《浯洲俗谚集》、吴家箴《浯岛情怀》、许丕华《浯乡俗谚风华录》、唐蕙韵整理《金门民间文学集.传说故事卷》。', []), ('2013年起，厦门大嶝岛与小嶝岛间填海造陆，越界盗采海砂，导致金门沿岸国土流失，古迹、碉堡设施损毁，海岸线侵蚀倒退情况严重。', []), ('几十年来，金门一直面临着居民供水困难的问题，原因包括湖水浅、雨量少、地理条件限制等，使得建水库和水坝难以实现。因此，金门经常过度使用地下水，导致潮洪上升，土壤盐碱化。', []), ('金门县政府在金门的教育上投入了数百万，平均每个学生两万元。福建的学校也接受越来越多父母在福建经商的台湾学生。县政府一直在努力鼓励台湾和大陆的大学在金门设立分校，并吸引大陆大陆学生到金门学习。', []), ('民国三十四年（1945年）第二次世界大战结束后，中华民国收复金门，设二镇四乡；民国35年（1946年）变更为二镇二乡。', [('民国三十四年（1945年）第二次世界大战结束后，中华民国收回金门，设二镇四乡；民国三十五年（1946年）变更为二镇二乡。', 

In [18]:
n_pairs = sum([len(pair) for pair in sent_pairs_b2w])
avg_pairs = n_pairs / len(sent_pairs_b2w)
print(n_pairs, avg_pairs)
print(sent_pairs_b2w[:10])

7002 2.0
[('民国三十四年（1945年）第二次世界大战结束后，中华民国收回金门，设二镇四乡；民国三十五年（1946年）变更为二镇二乡。', [('民国三十四年（1945年）第二次世界大战结束后，中华民国收复金门，设二镇四乡；民国35年（1946年）变更为二镇二乡。', 0.8484848484848485)]), ('御赐里琼林坊：琼林旧称平林，因贤才尽出故明熹宗天启五年被御赐为琼林。琼林村的蔡氏家族，是明朝中叶从河南开封迁徙而来，明熹宗天启五年，因为平林籍的进士蔡献臣赶走蛮夷有功，于是赐里名“琼林”。琼林家庙经文建会评其为“十四世宗祠”，为台湾地区“国家”二级古迹。', []), ('据史料记载：“鲁王为明太祖九世孙，名朱以海．京师既陷，转徙台州，张国雄等迎居绍兴，称鲁监国，督师江上，画钱塘而守。后为清兵所克，遁入海，依郑成功，辗转到金门，去监国号。成功初以礼待之，后渐懈，以海不能平，将往南澳，成功使人沉之海”；另据《辞海大事记》记载：康熙元年（1662年），鲁王薨于台湾，两处记载互相矛盾，还有待进一步考证。', []), ('2004年，金门县财政总收入1812637万元新台币；人均所得308202元新台币；人均生产总值329656元新台币。', []), ('金门属于亚热带季风气候，全年降雨多集中于四至八月，台风多生于七、八月，全年风向东风占8个月，每年五至八月为东南风及南风。因金门为在海峡中之岛屿，四面无高山屏障，中间则丘陵起伏，故风力较强，夏有西南海风的吹拂，每到清明时候常带来浓雾，台金交通常受影响；东有强烈的东北季风。', []), ('金门的地层，以花岗片麻岩为主，分布甚为广阔，约占总面积一半。岛上土壤概以砂土及裸露红壤土为代表。前者沙层厚、保水保肥力均差；后者表土薄、酸性重，腐植质少，皆不宜耕作，故岛上农作仅宜价值较低之耐旱性杂粮：如高粱、玉米、花生、番薯等。由于四面环海，浅滩深澳，鱼虾贝介类滋生，滨海居民乃讨生计于大海中，然因渔业资源有限，兼且幅员狭窄，地力贫瘠，雨量稀少，农产不丰，只有少量之杂粮与蔬菜。居民乃远渡重洋，谋生异域，或移居台澎，或远适南洋，金门华侨足迹遍布东南亚，人口总数达二十余万之众，自古就有“侨乡”之称。', []), ('由于发掘了鲁王真圹，在出土志里，说明旧时的谬误之处。据载王世系事迹綦详，卒年为王寅康熙元年（166

In [2]:
def split_data(pairs, maxlen=254, n_test=300, seed="ETST"):
    # remove long sentences
    _pairs = pairs
    pairs = []
    for src, targs in _pairs:
        if len(src) <= maxlen:
            targs = [(targ, score) for targ, score in targs if len(targ) <= maxlen]
            if targs:
                pairs.append((src, targs))
    
    random.seed(seed)
    random.shuffle(pairs)
    random.seed()
            
    sep1 = -2 * n_test
    sep2 = -n_test
    return pairs[:sep1], pairs[sep1:sep2], pairs[sep2:]

def pairs2datasets(pairs):
    train_pairs, valid_pairs, test_pairs = split_data(pairs)
    
    train_ds = []
    for src, targs in train_pairs:
        train_ds += [(src, targ) for targ, score in targs]
    
    valid_ds = [(src, targs[0][0]) for src, targs in valid_pairs if targs]
    test_ds = [(src, targs[0][0]) for src, targs in test_pairs if targs]
    return train_ds, valid_ds, test_ds

In [5]:
file = "/home/zchen/encyclopedia-text-style-transfer/data/ETST/sents_pairs_b2w.json"
sent_pairs_b2w = load_json(file)
file = "/home/zchen/encyclopedia-text-style-transfer/data/ETST/sents_pairs_w2b.json"
sent_pairs_w2b = load_json(file)

In [6]:
train_ds, valid_ds, test_ds = pairs2datasets(sent_pairs_b2w)
print(len(train_ds))
print(train_ds[:10])
print(len(valid_ds))
print(valid_ds[:10])
print(len(test_ds))
print(test_ds[:10])

815
[('中华人民共和国主席、副主席每届任期同全国人民代表大会每届任期相同。', '《中华人民共和国宪法》第六十七条规定，宪法的解释权属于全国人民代表大会常务委员会。'), ('中华人民共和国主席、副主席每届任期同全国人民代表大会每届任期相同。', '《中华人民共和国宪法 (1954年)》，简称五四宪法，是中华人民共和国的第一部宪法，1954年9月20日经第一届全国人民代表大会第一次会议审议通过，因其在1954年颁布，故称其为“五四宪法”。中华人民共和国现行的八二宪法（第四部宪法）就是建立在五四宪法的基础之上。'), ('中华人民共和国主席、副主席每届任期同全国人民代表大会每届任期相同。', '1978年《中华人民共和国宪法》，通称七八宪法或1978宪法，是1978年3月5日在第五届全国人民代表大会第一次会议上通过的《中华人民共和国宪法》。'), ('中华人民共和国主席、副主席每届任期同全国人民代表大会每届任期相同。', '1954年9月20日，第一届全国人民代表大会第一次会议上，代表们共投票1197张，同意票为1197张，全票通过《中华人民共和国宪法》。'), ('中华人民共和国主席、副主席每届任期同全国人民代表大会每届任期相同。', '《中华人民共和国宪法》是中华人民共和国的根本法，拥有最高的法律效力。'), ('中华人民共和国主席、副主席每届任期同全国人民代表大会每届任期相同。', '经历2年多的修订，1982年12月4日，第五届全国人民代表大会第五次会议审议通过新的宪法，成为中华人民共和国的第四部宪法。该宪法建立在五四宪法的基础上，构成了现今宪法的主体部分，其主要的修订内容包括：'), ('中华人民共和国主席、副主席每届任期同全国人民代表大会每届任期相同。', '新宪法于1978年3月5日在第五届全国人民代表大会第一次会议上通过。'), ('中华人民共和国主席、副主席每届任期同全国人民代表大会每届任期相同。', '经历2年多的修订，1982年12月4日，第五届全国人民代表大会第五次会议审议通过新的宪法，成为中华人民共和国的第四部宪法（简称“八二宪法”）。该宪法建立在五四宪法的基础上，构成了现今宪法的主体部分，其主要的修订内容包括：'), ('中华人民共和国主席、副主席每届任期同全国人民代表大会每届任期相同。', '《中华人民共和国宪法 (1982年)》，

In [18]:
def save_dataset(dataset, src_file, targ_file):
    src, targ = zip(*dataset)
    save_corpus(src, src_file)
    save_corpus(targ, targ_file)
    
def save_corpus(sent_list, file):
    corpus = '\n'.join(sent_list) + '\n'
    with open(file, 'w', encoding="utf-8") as f:
        f.write(corpus)
    
for dataset, split in [(train_ds, "train"), (valid_ds, "valid"), (test_ds, "test")]:
    src_file = f"/home/zchen/XLM_ETST/data/baidu-wiki/txt/baidu-wiki.baidu.{split}_raw"
    targ_file = f"/home/zchen/XLM_ETST/data/baidu-wiki/txt/baidu-wiki.wiki.{split}_raw"
    save_dataset(dataset, src_file, targ_file)

In [6]:
def print_word_pos_sentence(word_sentence, pos_sentence):
    assert len(word_sentence) == len(pos_sentence)
    for word, pos in zip(word_sentence, pos_sentence):
        print(f"{word}({pos})", end="\u3000")
    print()
    return
    
for i, sentence in enumerate(sentence_list):
    print()
    print(f"'{sentence}'")
    print_word_pos_sentence(word_sentence_list[i],  pos_sentence_list[i])
    for entity in sorted(entity_sentence_list[i]):
        print(entity)

NameError: name 'sentence_list' is not defined